In [76]:
from torch import nn
import torch
from torch.nn.utils.rnn import pack_sequence
import torch.optim as optim
from tqdm import tqdm

In [77]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.l1 = nn.RNN(256,40,1, batch_first=True)
        self.l2 = nn.Linear(40, 256)

    def forward(self, input):
        _, l1o = self.l1(input)
        output = self.l2(l1o)
        return output

model = MyModel()

In [78]:
enwik9 = open('enwik9', 'rb').read()

In [79]:
device = torch.device('cpu')

In [80]:
x = torch.randn(1, 10, device=device)
y = torch.randn(1, 1, device=device)

In [81]:
for i in range(20, 100):#len(enwik9)
    text_in = enwik9[i-20:i]
    char_out = enwik9[i+1]

In [82]:
def byte_to_tensor(c):
    tensor = torch.zeros(256)
    tensor[c] = 1
    return tensor

def bytes_to_tensor(b):
    tensor = torch.zeros(len(b), 256)
    for idx, c in enumerate(b):
        tensor[idx][c] = 1
    return tensor

In [83]:
t = bytes_to_tensor(b'abcabcabca')
x = torch.reshape(t, (1,10,256))
model.l1(x)

(tensor([[[ 0.1594,  0.2814, -0.2901,  0.0052,  0.0497,  0.2934,  0.1070,
            0.1368,  0.1765, -0.1226,  0.1894, -0.0215, -0.0437,  0.0323,
            0.2266, -0.0776,  0.1209, -0.0064,  0.0033, -0.2500, -0.0315,
            0.1000,  0.1812,  0.1018, -0.0332,  0.3113, -0.2247,  0.1814,
           -0.2251, -0.0967,  0.2895, -0.1670, -0.1737,  0.1420, -0.1413,
            0.0784, -0.1128, -0.1471, -0.1236, -0.2602],
          [ 0.0547,  0.3182, -0.2519,  0.0331, -0.2086,  0.3373,  0.2168,
           -0.0199,  0.1638, -0.0727, -0.0028, -0.0366, -0.0833,  0.0534,
            0.0896,  0.2121,  0.4529,  0.0595, -0.1920, -0.2421,  0.1143,
            0.1770,  0.1872,  0.0940, -0.0650,  0.1360, -0.0988,  0.0857,
           -0.0363, -0.1746,  0.2555, -0.1206,  0.3261,  0.2186, -0.0048,
            0.1349,  0.0097, -0.3780, -0.2607,  0.0974],
          [-0.1595,  0.1698, -0.1605,  0.1026, -0.0610,  0.3749,  0.0339,
            0.1156,  0.0995,  0.1777,  0.1848,  0.1159, -0.2118, -0.0933

In [93]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for i in range(20):
    total_loss = 0
    for i in tqdm(range(20, 100000)):#len(enwik9)
        optimizer.zero_grad()
        text_in = enwik9[i-10:i]
        char_out = enwik9[i+1]
        t_x = bytes_to_tensor(text_in)
        t_x = torch.reshape(t_x, (1,10,256))
        t_y = byte_to_tensor(char_out)
        y = model(t_x)
        y = torch.reshape(y, (1,256))
        loss = criterion(y, torch.tensor([char_out]))
        total_loss += loss
        loss.backward()
        optimizer.step()    # Does the update
    print(total_loss)

  0%|          | 0/99980 [00:00<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [91]:
torch.argmax(model(torch.reshape(bytes_to_tensor(b'this is a message. whe'[-10:]), (1,10,256))))

tensor(99)

In [92]:
chr(99)

'c'